In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.4` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:0.10.9` 

In [6]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


import org.apache.log4j.{Level, Logger}


In [7]:
import org.apache.spark.sql._


import org.apache.spark.sql._


In [8]:
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .config("spark.testing.memory", 471859200)
    .config("spark.executor.instances", "4")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
}

spark: SparkSession = org.apache.spark.sql.SparkSession@5d5c2b7f

In [9]:
val df = spark.read.format("parquet").load("data/train")

df: DataFrame = [instanceId_userId: int, instanceId_objectType: string ... 167 more fields]